In [1]:
import os
import tensorflow as tf
tf.enable_eager_execution()

import math
import numpy as np
import itertools

In [2]:
from waymo_open_dataset.utils import range_image_utils
from waymo_open_dataset.utils import transform_utils
from waymo_open_dataset.utils import  frame_utils
from waymo_open_dataset import dataset_pb2 as open_dataset

In [3]:
def scantext():                                                       #text files of VR data
    x=[]
    for path,dirs,files in os.walk('./waymo-od/data'):
        for f in files:
            if f.startswith('segment'):
                x.append(os.path.join(path,f))
    return(x)



alltext=scantext()

In [4]:
alltext

['./waymo-od/data/training_0000/segment-10153695247769592104_787_000_807_000_with_camera_labels.tfrecord',
 './waymo-od/data/training_0000/segment-10231929575853664160_1160_000_1180_000_with_camera_labels.tfrecord',
 './waymo-od/data/training_0000/segment-10075870402459732738_1060_000_1080_000_with_camera_labels.tfrecord',
 './waymo-od/data/training_0000/segment-10023947602400723454_1120_000_1140_000_with_camera_labels.tfrecord',
 './waymo-od/data/training_0000/segment-10082223140073588526_6140_000_6160_000_with_camera_labels.tfrecord',
 './waymo-od/data/training_0000/segment-10094743350625019937_3420_000_3440_000_with_camera_labels.tfrecord',
 './waymo-od/data/training_0000/segment-10072231702153043603_5725_000_5745_000_with_camera_labels.tfrecord',
 './waymo-od/data/training_0000/segment-10072140764565668044_4060_000_4080_000_with_camera_labels.tfrecord',
 './waymo-od/data/training_0000/segment-10235335145367115211_5420_000_5440_000_with_camera_labels.tfrecord',
 './waymo-od/data/tra